## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jun. 24th Jul. 1st


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [8]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-06-10') & (df['SOLD DATE'] <= '2024-06-17')]

In [7]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
146,PAST SALE,2024-07-01,Condo/Co-op,480 NE 31st St #1101,Miami,FL,33137.0,1700000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/480-NE-31st-St...,MARMLS,A11519633,N,Y,25.806186,-80.187176
5,PAST SALE,2024-06-29,Condo/Co-op,18555 Collins Ave #3003,Sunny Isles Beach,FL,33160.0,3650000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11611938,N,Y,25.948195,-80.120410
71,PAST SALE,2024-06-29,Condo/Co-op,18683 Collins Ave #610,Sunny Isles Beach,FL,33160.0,635000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11548133,N,Y,25.949174,-80.119837
96,PAST SALE,2024-06-28,Condo/Co-op,501 NE 31st St #2306,Miami,FL,33137.0,755000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/501-NE-31st-St...,MARMLS,A11512036,N,Y,25.806882,-80.187314
88,PAST SALE,2024-06-28,Condo/Co-op,7805 Camino Real Unit H215,Miami,FL,33143.0,247000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/7805-Camino-Re...,MARMLS,A11571566,N,Y,25.692826,-80.320229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,PAST SALE,2024-06-24,Condo/Co-op,7011 Crown Gate Ct #7011,Miami Lakes,FL,33014.0,525000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Lakes/7011-Cro...,MARMLS,A11538331,N,Y,25.914293,-80.315032
17,PAST SALE,2024-06-24,Condo/Co-op,8895 Fontainebleau Blvd #109,Miami,FL,33172.0,290000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/8895-Fontaineb...,MARMLS,A11438740,N,Y,25.776029,-80.338280
142,PAST SALE,2024-06-24,Condo/Co-op,1663 SE 29th St #209,Homestead,FL,33035.0,215000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/1663-SE-29...,MARMLS,A11514962,N,Y,25.443242,-80.453364
144,PAST SALE,2024-06-24,Condo/Co-op,2461 SE 16th Ter #203,Homestead,FL,33035.0,215000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Homestead/2461-SE-16...,MARMLS,A11496968,N,Y,25.447178,-80.453598


In [8]:
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [9]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

190


In [10]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$748,529


In [11]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$517


In [12]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$142,220,489


In [54]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [13]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
147
Input Previous Week Condo Average Sales Price:
1_120_000
Input Previous Week Condo Average PSF:
598
Input Previous Week Condo Sales total (ex: 198_000_000)
172_000_000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [14]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [15]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [16]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [17]:
pd.set_option('display.max_columns',None)

In [18]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
173,PAST SALE,2024-06-25,Condo/Co-op,1 Collins Ave Ph 706,Miami Beach,FL,33139.0,10000000.0,4.0,5.5,ONE OCEAN CONDO,3339.0,NaN,2016.0,NaN,2995.0,8131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1-Collin...,MARMLS,A11454047,N,Y,25.769375,-80.133358,1,orange


In [19]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [20]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [21]:
m.save('index.html')

## Data snagger

In [22]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [23]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
173,PAST SALE,2024-06-25,Condo/Co-op,1 Collins Ave Ph 706,Miami Beach,FL,33139.0,10000000.0,4.0,5.5,ONE OCEAN CONDO,3339.0,NaN,2016.0,NaN,2995.0,8131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1-Collin...,MARMLS,A11454047,N,Y,25.769375,-80.133358,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [24]:
df_top_ten['PRICE'].iloc[0]

10000000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [25]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_07012024


# CREATE TEMPLATE 

In [26]:
muni_set = set(df_top_ten['CITY'])

In [27]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [28]:
df_top_ten.reset_index(inplace=True,drop=True)

In [29]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [30]:
top_sale

'ONE OCEAN CONDO at 1 Collins Ave Ph 706 in Miami Beach'

In [31]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [111]:
# df_top_ten

## RESUME HERE

In [32]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: ONE OCEAN CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $3,000,000M to $10,000,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami Beach, Sunny Isles Beach, Key Biscayne, Bal Harbour, Miami

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 190 condo sales totaling $142,220,489 million from Jun. 24th to Jul. 1st. The previous week, brokers closed 147 condo sales totaling $172,000,000.

Last week’s units sold for an average of $748,529, lower than the $1,120,000 sales average from the previous week. The 

In [33]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [34]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [35]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: ONE OCEAN CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $3,000,000M to $10,000,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami Beach, Sunny Isles Beach, Key Biscayne, Bal Harbour, Miami

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 190 condo sales totaling $142,220,489 million from Jun. 24th to Jul. 1st. The previous week, brokers closed 147 condo sales totaling $172,000,000.

Last week’s units sold for an average of $748,529, lower than the $1,120,000 sales average from the previous week. The 

In [45]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Key-Biscayne/360-Ocean-Dr-33149/unit-301S/home/79133307


In [46]:
print(df_top_ten['URL'].iloc[2])

https://www.redfin.com/FL/Miami/788-NE-23rd-St-33137/unit-5102/home/174545668


In [37]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Bal-Harbour/9601-Collins-Ave-33154/unit-1009/home/42895625


In [38]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/North-Miami-Beach/15701-Collins-Ave-33160/unit-2804/home/171036233


In [39]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/1-Collins-Ave-33139/unit-706/home/118777606


In [43]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/1643-Brickell-Ave-33129/unit-3204/home/42759346


In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-06-25,Condo/Co-op,1 Collins Ave Ph 706,Miami Beach,FL,33139.0,10000000.0,4.0,5.5,ONE OCEAN CONDO,3339.0,NaN,2016.0,NaN,2995.0,8131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1-Collin...,MARMLS,A11454047,N,Y,25.769375,-80.133358,1,orange,ONE OCEAN CONDO at 1 Collins Ave Ph 706 in Mia...
1,PAST SALE,2024-06-24,Condo/Co-op,360 Ocean Dr Unit 301S,Key Biscayne,FL,33149.0,8500000.0,4.0,5.5,OCEANA KEY BISCAYNE CONDO,3798.0,NaN,2014.0,NaN,2238.0,4782.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/360-Oce...,MARMLS,A11532438,N,Y,25.694584,-80.157715,2,blue,OCEANA KEY BISCAYNE CONDO at 360 Ocean Dr Unit...
2,PAST SALE,2024-06-25,Condo/Co-op,788 NE 23rd St #5102,Miami,FL,33137.0,4500000.0,4.0,4.5,Elysee,3488.0,NaN,2021.0,NaN,1290.0,4175.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/788-NE-23rd-St...,MARMLS,A11525740,N,Y,25.799065,-80.185542,3,blue,Elysee at 788 NE 23rd St #5102 in Miami
3,PAST SALE,2024-06-25,Condo/Co-op,1643 Brickell Ave #3204,Miami,FL,33129.0,4470000.0,4.0,5.5,SANTA MARIA CONDO,4030.0,NaN,1997.0,NaN,1109.0,4592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11542639,N,Y,25.755513,-80.195018,4,blue,SANTA MARIA CONDO at 1643 Brickell Ave #3204 i...
4,PAST SALE,2024-06-28,Condo/Co-op,6899 Collins Ave #404,Miami Beach,FL,33141.0,4400000.0,2.0,3.5,NORTH CARILLON BEACH COND,3105.0,NaN,2008.0,NaN,1417.0,7977.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6899-Col...,MARMLS,A11449086,N,Y,25.853732,-80.120089,5,blue,NORTH CARILLON BEACH COND at 6899 Collins Ave ...
5,PAST SALE,2024-06-25,Condo/Co-op,781 Crandon Blvd #1504,Key Biscayne,FL,33149.0,4000000.0,3.0,4.5,CLUB TOWER THREE CONDO,2500.0,NaN,1999.0,NaN,1600.0,3630.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/781-Cra...,MARMLS,A11431154,N,Y,25.688277,-80.158928,6,blue,CLUB TOWER THREE CONDO at 781 Crandon Blvd #15...
6,PAST SALE,2024-06-29,Condo/Co-op,18555 Collins Ave #3003,Sunny Isles Beach,FL,33160.0,3650000.0,4.0,4.5,PORSHE DESING TOWER,3171.0,NaN,2016.0,NaN,1151.0,6250.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11611938,N,Y,25.948195,-80.120410,7,blue,PORSHE DESING TOWER at 18555 Collins Ave #3003...
7,PAST SALE,2024-06-24,Condo/Co-op,785 Crandon Blvd #1604,Key Biscayne,FL,33149.0,3625000.0,3.0,3.5,CLUB TOWER TWO CONDO,2500.0,NaN,1998.0,NaN,1450.0,3631.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/785-Cra...,MARMLS,A11435848,N,Y,25.687567,-80.159386,8,blue,CLUB TOWER TWO CONDO at 785 Crandon Blvd #1604...
8,PAST SALE,2024-06-28,Condo/Co-op,15701 Collins Ave #2804,Sunny Isles Beach,FL,33160.0,3400000.0,2.0,2.5,15701 COLLINS CONDO,1571.0,NaN,2020.0,NaN,2164.0,2676.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11572724,N,Y,25.921944,-80.121716,9,blue,15701 COLLINS CONDO at 15701 Collins Ave #2804...
9,PAST SALE,2024-06-25,Condo/Co-op,9601 Collins #1009,Bal Harbour,FL,33154.0,3000000.0,2.0,2.5,MAJESTIC TOWER,2160.0,NaN,1998.0,NaN,1389.0,2502.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9601-Col...,MARMLS,A11586201,N,Y,25.887706,-80.122180,10,blue,MAJESTIC TOWER at 9601 Collins #1009 in Bal Ha...


## Time on Market Calculator

In [60]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 11, 23) ## List (Earlier) date
date2 = datetime(2024, 6, 21) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

211
